In [1]:
import numpy as np
import time
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import cv2
import os
from matplotlib import pyplot as plt
from skimage.color import rgb2gray
from skimage.transform import warp
from skimage.registration import optical_flow_tvl1, optical_flow_ilk
import openslide
import os
import cv2
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import skimage.measure
import scipy.stats as stats
import torchvision.transforms as transforms
from glob import glob
from time import time
from skimage.measure import label

In [2]:
def _time(f):
    def wrapper(*args,**kwargs):
        start=time()
        r=f(*args,**kwargs)
        end=time()
        print("%s timed %f" %(f.__name__,end-start))
        return r
    return wrapper

In [3]:
# First, input xml_filepath and output a dataframe of X,Y coordinates in general. (can be used for ROI as well)
def xml_to_df(xml_filepath):
    tree = ET.parse(xml_filepath)
    root = tree.getroot()
    append_df = []
    for index, Annotation in enumerate(root.iter("Annotation")):
        for Region in Annotation.iter('Region'):
            x = np.array([Vertex.get('X') for Vertex in Region.iter('Vertex')])
            y = np.array([Vertex.get('Y') for Vertex in Region.iter('Vertex')])
            id = np.array([int(Region.get('Id'))])
            classnames = index + 1
            coord_dict = {"ClassNames": [classnames], "X": [x], "Y": [y], "ID": [id]}
            df = pd.DataFrame(data = coord_dict)
            df.ID = df.ID.astype(int)
            append_df.append(df)
    coord_df = pd.concat(append_df).reset_index(drop=True)
    return(coord_df)

In [4]:
coord_df = xml_to_df(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\sk244a.xml')
coord_df

,ClassNames,X,Y,ID
0,1,"[66855, 66848, 66848, 66848, 66848, 66848, 668...","[29667, 29667, 29674, 29681, 29688, 29695, 297...",1
1,1,"[64319, 64382, 64395, 64421, 64446, 64472, 644...","[63336, 63399, 63412, 63438, 63451, 63476, 635...",2
2,1,"[78061, 78048, 78048, 78048, 78048, 78048, 780...","[28813, 28813, 28851, 28915, 29005, 29068, 291...",3
3,1,"[66800, 66791, 66781, 66763, 66754, 66735, 667...","[38608, 38608, 38617, 38636, 38654, 38682, 387...",4
4,1,"[66050, 66060, 66078, 66106, 66143, 66180, 661...","[49432, 49422, 49404, 49395, 49367, 49330, 493...",5
5,1,"[78649, 78649, 78649, 78649, 78649, 78649, 786...","[38239, 38252, 38264, 38290, 38328, 38354, 383...",6
6,1,"[77691, 77678, 77665, 77639, 77627, 77601, 775...","[48635, 48635, 48635, 48648, 48661, 48686, 487...",7
7,1,"[78163, 78291, 78432, 78559, 78738, 78879, 790...","[67984, 67984, 67997, 67997, 67997, 67997, 679...",8
8,1,"[89727, 89727, 89713, 89699, 89693, 89679, 896...","[66995, 66954, 66905, 66863, 66821, 66793, 667...",9
9,1,"[88630, 88593, 88574, 88537, 88519, 88500, 884...","[55128, 55054, 54999, 54906, 54851, 54795, 547...",10


### for stained: downsample by 8x to find the bounding box for each tissue image in WSI, and then get those boundingbox coordsinates multipled by 8x again, and use those coordinates to read region with no downsample, but only that region of interest and then crop. (modify read_region function to read at full resolution, but only the bounding box by changing location and size).

### for unstained: use xml coordinates and just find the boudnign box from there and then read region with no downsample and crop

In [111]:
# for unstained:
def unstained_crop(xml_filepath, unstain_ndpi_filepath):
    coord_df = xml_to_df(xml_filepath)
    slide = openslide.open_slide(unstain_ndpi_filepath)
    rgb_dim = slide.dimensions
    print("Dimension of level 0 of the unstained image is: ",rgb_dim)
    slide_level_dim = slide.level_dimensions
    num_levels = len(slide_level_dim)
    print("Number of levels in this unstained image are:",num_levels)
    factors = slide.level_downsamples
    print("For unstained image, each level is downsampled by:",factors)
    tmp = os.path.basename(xml_filepath)
    ndpi_name = os.path.splitext(tmp)[0]

    for idx, row in coord_df.iterrows():
        xx = row.X.astype('int')
        yy = row.Y.astype('int')
        x_min = np.min(xx)
        x_max = np.max(xx)
        y_min = np.min(yy)
        y_max = np.max(yy)
        location = (x_min,y_min)
        dim = (x_max-x_min,y_max-y_min)
        image_resized = slide.read_region(location= location,level=0,size=dim)
        image_resized = np.array(image_resized)
        final_img = Image.fromarray(image_resized)
        save_path = os.path.join(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\images',ndpi_name)
        save_path = os.path.join(save_path, ndpi_name + str(idx) + ".png")
        final_img.save(save_path)

    print("Images saved successfully!")

In [ ]:
unstained_crop(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\sk244a.xml', r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\sk244a.ndpi')

Dimension of level 0 of the unstained image is:  (199680, 98560)
Number of levels in this unstained image are: 8
For unstained image, each level is downsampled by: (1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0)


In [ ]:
# for stained:

# coord_df = xml_to_df(xml_filepath)
# slide = openslide.open_slide(unstain_ndpi_filepath)
# rgb_dim = slide.dimensions
# print("Dimension of level 0 of the unstained image is: ",rgb_dim)
# slide_level_dim = slide.level_dimensions
# num_levels = len(slide_level_dim)
# print("Number of levels in this unstained image are:",num_levels)
# factors = slide.level_downsamples
# print("For unstained image, each level is downsampled by:",factors)
# target_level = slide.get_best_level_for_downsample(downsample_factor)
# target_dim = slide.level_dimensions[target_level]
# rsf = [x/y for x,y in zip(rgb_dim,target_dim)]
# image_resized = slide.read_region(location=(0,0),level=target_level,size=target_dim)
# image_resized = np.array(image_resized)
# print("unstained image size after downsample is: {}".format(image_resized.shape))
# tmp = os.path.basename(xml_filepath)
# ndpi_name = os.path.splitext(tmp)[0]
# for idx, row in coord_df.iterrows():
#     xx = row.X.astype('int')
#     yy = row.Y.astype('int')
#     x_min = round(np.min(xx)/rsf[0])
#     x_max = round(np.max(xx)/rsf[0])
#     y_min = round(np.min(yy)/rsf[0])
#     y_max = round(np.max(yy)/rsf[0])
#     final_img = image_resized[y_min:y_max,x_min:x_max]
#     final_img = Image.fromarray(final_img)
#     save_path = os.path.join(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\images',ndpi_name)
#     save_path = os.path.join(save_path, ndpi_name + str(idx) + ".png")
#     final_img.save(save_path)
#     print("Images saved successfully!")

In [3]:
# function to pad images to same size:
def pad_images_to_same_size(images):
    """
    :param images: sequence of images
    :return: list of images padded so that all images have same width and height (max width and height are used)
    """
    width_max = 0
    height_max = 0
    for img in images:
        h, w = img.shape[:2]
        width_max = max(width_max, w)
        height_max = max(height_max, h)

    images_padded = []
    for img in images:
        h, w = img.shape[:2]
        diff_vert = height_max - h
        pad_top = diff_vert//2
        pad_bottom = diff_vert - pad_top
        diff_hori = width_max - w
        pad_left = diff_hori//2
        pad_right = diff_hori - pad_left
        img_padded = cv2.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=(255,255,255))
        assert img_padded.shape[:2] == (height_max, width_max)
        images_padded.append(img_padded)

    return images_padded

In [4]:
pad_images_to_same_size = _time(pad_images_to_same_size)
optical_flow_tvl1 = _time(optical_flow_tvl1)

## First create separate images from each wsi image, with annotations in the unstained folder. crop the images (total of 24 annotation, so 24 images per whole slide) and save them first in a different folder in same path. Then pad the images and then save them. Then using those padded images, register them to one another (doesn't matter which way since there are only two images)
## the path is in: \\shelter\Kyu\unstain2stain\biomax_images

In [ ]:
def registrate_two_images(reference_image_path, moving_image_path, save_path):
    ref_img_path = [_ for _ in os.listdir(reference_image_path) if _.endswith(".jpg")]
    ref_img_path_complete = [os.path.join(ref_img_path, x) for x in ref_img_path]
    ref_img_path_1 = [x.replace('.jpg','') for x in ref_img_path]

    mov_img_path = [_ for _ in os.listdir(moving_image_path) if _.endswith(".jpg")]
    mov_img_path_complete = [os.path.join(mov_img_path, x) for x in mov_img_path]
    mov_img_path_1 = [x.replace('.jpg','') for x in mov_img_path]

    if int(len(ref_img_path)) != int(len(mov_img_path)):
        raise

    num = int(len(ref_img_path)/2) - 1 #idx = 16, or 17th image
    num_plus1 = num + 1 #idx = 17, or 18th image
    num_minus1 = num - 1 #idx = 15, or 16th image

    start = time()

    ref_img_path = img_files_path_complete[num]
    mov_img_path = img_files_path_complete[num_plus1]
    ref_img = np.array(Image.open(ref_img_path))
    mov_img = np.array(Image.open(mov_img_path))

    ref_img_g = rgb2gray(ref_img)
    mov_img_g = rgb2gray(mov_img)
    v, u = optical_flow_tvl1(ref_img_g, mov_img_g)
    nr, nc = ref_img_g.shape
    row_coords, col_coords = np.meshgrid(np.arange(nr), np.arange(nc),
                                         indexing='ij')
    end = time()

    start = time()

    mov_img_warp_ra =[]
    for i in range(3):
        mov_img_warp = warp(mov_img[:,:,i], np.array([row_coords + v, col_coords + u]),mode='edge')
        mov_img_warp_ra.append(mov_img_warp)

    r = np.array(mov_img_warp_ra[0]*255).astype('uint8')
    g = np.array(mov_img_warp_ra[1]*255).astype('uint8')
    b = np.array(mov_img_warp_ra[2]*255).astype('uint8')
    rgb = np.stack([r,g,b],axis=2)
    reg_img = Image.fromarray(rgb)
    reg_img.save(r'\\fatherserverdw\kyuex\image\CLUE\3D study\he\CoarseRegIM\run4_16xr_jpg_rszfc1_padsz1000\opticalflow_registered_image\\' + str(img_files_path_1[num_plus1]) + '.jpg')

    end = time()
    print("time it took to register: "+  str(end-start) + " seconds")


In [ ]:
#two for loops to recursively create registered images:

# from middle to end (higher index), idx of 17 to 33, or 18th image to 34th image:
start = time()

for idx in range(num_plus1,len(img_files_path_complete)): #idx = 17 to 34 (not including 34)
    if idx == len(img_files_path_complete) - 1:
        break
    ref_img_path = os.path.join(r'\\fatherserverdw\kyuex\image\CLUE\3D study\he\CoarseRegIM\run4_16xr_jpg_rszfc1_padsz1000\opticalflow_registered_image', img_files_path[idx])
    mov_img_path = img_files_path_complete[idx+1]
    ref_img = np.array(Image.open(ref_img_path))
    mov_img = np.array(Image.open(mov_img_path))

    ref_img_g = rgb2gray(ref_img)
    mov_img_g = rgb2gray(mov_img)
    v, u = optical_flow_tvl1(ref_img_g, mov_img_g)
    nr, nc = ref_img_g.shape
    row_coords, col_coords = np.meshgrid(np.arange(nr), np.arange(nc),
                                         indexing='ij')
    mov_img_warp_ra =[]
    for i in range(3):
        mov_img_warp = warp(mov_img[:,:,i], np.array([row_coords + v, col_coords + u]),mode='edge')
        mov_img_warp_ra.append(mov_img_warp)
    r = np.array(mov_img_warp_ra[0]*255).astype('uint8')
    g = np.array(mov_img_warp_ra[1]*255).astype('uint8')
    b = np.array(mov_img_warp_ra[2]*255).astype('uint8')
    rgb = np.stack([r,g,b],axis=2)
    reg_img = Image.fromarray(rgb)
    reg_img.save(r'\\fatherserverdw\kyuex\image\CLUE\3D study\he\CoarseRegIM\run4_16xr_jpg_rszfc1_padsz1000\opticalflow_registered_image\\' + str(img_files_path_1[idx+1]) + '.jpg')

end = time()
print("time it took to register: "+  str((end-start)/60) + " minutes")